In [8]:
# pip install configparser

In [1]:
import boto3
import psycopg2
import pandas as pd 
import json
import configparser

In [2]:
config=configparser.ConfigParser()
config.read_file(open('cluster.config'))

In [3]:
config.get('AWS','KEY')

'****************************'

In [31]:

KEY=                    config.get('AWS','KEY')
SECRET=                 config.get('AWS','SECRET')


DWH_CLUSTER_TYPE=       config.get('DWH','DWH_CLUSTER_TYPE')
DWH_NUM_NODES=          config.get('DWH','DWH_NUM_NODES')
DWH_NODE_TYPE=           config.get('DWH','DWH_NODE_TYPE')

DWH_CLUSTER_IDENTIFIER= config.get('DWH','DWH_CLUSTER_IDENTIFIER')
DWH_DB=                 config.get('DWH','DWH_DB')
DWH_DB_USER=            config.get('DWH','DWH_DB_USER')
DWH_DB_PASSWORD=        config.get('DWH','DWH_DB_PASSWORD')
DWH_PORT=               config.get('DWH','DWH_PORT')

DWH_IAM_ROLE_NAME=      config.get('DWH','DWH_IAM_ROLE_NAME')

(DWH_DB_USER,DWH_DB_PASSWORD,DWH_DB)

('awsuser', 'Passw0rd123', 'myfirstdb')

In [32]:
pd.DataFrame({"Param":
                    ['DWH_CLUSTER_TYPE','DWH_NUM_NODES','DWH_NODE_TYPE','DWH_CLUSTER_IDENTIFIER','DWH_DB','DWH_DB_USER','DWH_DB_PASSWORD','DWH_PORT','DWH_IAM_ROLE_NAME'],

                "Value":
                [DWH_CLUSTER_TYPE,DWH_NUM_NODES,DWH_NODE_TYPE,DWH_CLUSTER_IDENTIFIER,DWH_DB,DWH_DB_USER,DWH_DB_PASSWORD,DWH_PORT,DWH_IAM_ROLE_NAME]
                  
})

,Param,Value
0,DWH_CLUSTER_TYPE,single-node
1,DWH_NUM_NODES,1
2,DWH_NODE_TYPE,dc2.large
3,DWH_CLUSTER_IDENTIFIER,my-first-redshift-new
4,DWH_DB,myfirstdb
5,DWH_DB_USER,awsuser
6,DWH_DB_PASSWORD,Passw0rd123
7,DWH_PORT,5439
8,DWH_IAM_ROLE_NAME,redshift-s3-acces


In [33]:
ec2=boto3.resource('ec2',
                region_name='us-west-2',
                aws_access_key_id=KEY,
                aws_secret_access_key=SECRET
)

In [34]:


s3=boto3.resource('s3',
                region_name='us-west-2',
                aws_access_key_id=KEY,
                aws_secret_access_key=SECRET
)

iam=boto3.client('iam',
                region_name='us-west-2',
                aws_access_key_id=KEY,
                aws_secret_access_key=SECRET
)

redshift=boto3.client('redshift',
                region_name='us-west-2',
                aws_access_key_id=KEY,
                aws_secret_access_key=SECRET
)

In [35]:

bucket=s3.Bucket('mabohassan-test')
log_data_files=[filename.key for filename in bucket.objects.filter(Prefix='')]
log_data_files

['allevents_pipe.txt',
 'allusers_pipe.txt',
 'category_pipe.txt',
 'date2008_pipe.txt',
 'listings_pipe.txt',
 'sales_tab.txt',
 'venue_pipe.txt']

In [36]:
roleArn=iam.get_role(RoleName=DWH_IAM_ROLE_NAME)['Role']['Arn']
roleArn

'arn:aws:iam::777381563271:role/redshift-s3-acces'

In [37]:

try:
    response=redshift.create_cluster(
    ClusterType=DWH_CLUSTER_TYPE,
    NodeType=DWH_NODE_TYPE,
    
    #identifiers & credentials
    DBName=DWH_DB,
    ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,
    MasterUsername=DWH_DB_USER,
    MasterUserPassword=DWH_DB_PASSWORD,

    #roles (for s3 access)
    IamRoles=[roleArn]


    )
except Exception as e:
    print(e)

In [4]:
# redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]

In [39]:
def prettyRedshiftProps(props):
    pd.set_option('display.max_colwidth', -1)
    keysToShow = ["ClusterIdentifier", "NodeType", "ClusterStatus", "MasterUsername", "DBName", "Endpoint", "NumberOfNodes", 'VpcId']
    x = [(k, v) for k,v in props.items() if k in keysToShow]
    return pd.DataFrame(data=x, columns=["Key", "Value"])
myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

c:\Users\abohassan\anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


,Key,Value
0,ClusterIdentifier,my-first-redshift-new
1,NodeType,dc2.large
2,ClusterStatus,available
3,MasterUsername,awsuser
4,DBName,myfirstdb
5,Endpoint,"{'Address': 'my-first-redshift-new.ctfdnxkjcgk8.us-west-2.redshift.amazonaws.com', 'Port': 5439}"
6,VpcId,vpc-032e0f5e07d1abf94
7,NumberOfNodes,1


In [40]:
myClusterProps.keys()

dict_keys(['ClusterIdentifier', 'NodeType', 'ClusterStatus', 'ClusterAvailabilityStatus', 'MasterUsername', 'DBName', 'Endpoint', 'ClusterCreateTime', 'AutomatedSnapshotRetentionPeriod', 'ManualSnapshotRetentionPeriod', 'ClusterSecurityGroups', 'VpcSecurityGroups', 'ClusterParameterGroups', 'ClusterSubnetGroupName', 'VpcId', 'AvailabilityZone', 'PreferredMaintenanceWindow', 'PendingModifiedValues', 'ClusterVersion', 'AllowVersionUpgrade', 'NumberOfNodes', 'PubliclyAccessible', 'Encrypted', 'ClusterPublicKey', 'ClusterNodes', 'ClusterRevisionNumber', 'Tags', 'EnhancedVpcRouting', 'IamRoles', 'MaintenanceTrackName', 'DeferredMaintenanceWindows', 'NextMaintenanceWindowStartTime', 'AvailabilityZoneRelocationStatus', 'ClusterNamespaceArn', 'AquaConfiguration'])

In [41]:
DWH_ENDPOINT=myClusterProps['Endpoint']['Address']
DWH_ROLE_ARN=myClusterProps['IamRoles'][0]['IamRoleArn']
DB_NAME=myClusterProps['DBName']
DB_USER=myClusterProps['MasterUsername']

In [42]:
DWH_ENDPOINT

'my-first-redshift-new.ctfdnxkjcgk8.us-west-2.redshift.amazonaws.com'

In [43]:
DB_USER

'awsuser'

In [44]:
#Opening a TCP port to access the cluster endpoint
try:
    vpc = ec2.Vpc(id=myClusterProps['VpcId'])
    defaultSg = list(vpc.security_groups.all())[0]
    print(defaultSg)
    
    defaultSg.authorize_ingress(
        GroupName= defaultSg.group_name,
        CidrIp='0.0.0.0/0',
        IpProtocol='TCP',
        FromPort=int(DWH_PORT),
        ToPort=int(DWH_PORT)
    )
except Exception as e:
    print(e)

ec2.SecurityGroup(id='sg-0fb06e44e5383522c')
An error occurred (InvalidPermission.Duplicate) when calling the AuthorizeSecurityGroupIngress operation: the specified rule "peer: 0.0.0.0/0, TCP, from port: 5439, to port: 5439, ALLOW" already exists


In [45]:
DWH_PORT

'5439'

In [46]:
try:
    conn=psycopg2.connect(host=DWH_ENDPOINT,dbname=DWH_DB,user=DWH_DB_USER, password=DWH_DB_PASSWORD,  port=DWH_PORT,)
except psycopg2.Error as e:
    print('could not connect')
    print(e)

conn.set_session(autocommit=True)

In [47]:
try: 
    cur = conn.cursor()
except psycopg2.Error as e: 
    print("Error: Could not get curser to the Database")
    print(e)

In [49]:
try:
    cur.execute("""create table users(
        userid integer not null distkey sortkey,
        username char(8),
        firstname varchar(30),
        lastname varchar(30),
        city varchar(30),
        state char(2),
        email varchar(100),
        phone char(14),
        likesports boolean,
        liketheatre boolean,
        likeconcerts boolean,
        likejazz boolean,
        likeclassical boolean,
        likeopera boolean,
        likerock boolean,
        likevegas boolean,
        likebroadway boolean,
        likemusicals boolean);""")
except psycopg2.Error as e:
    print('Error: issue creating table')
    print(e)

In [50]:
try:
    cur.execute("""create table venue(
        venueid smallint not null distkey sortkey,
        venuename varchar(100),
        venuecity varchar(30),
        venuestate char(2),
        venueseats integer);""")
except psycopg2.Error as e:
        print('Error: issue creating table')
        print(e)

In [53]:
try:
    cur.execute("""
        create table category(
            catid smallint not null distkey sortkey,
            catgroup varchar(10),
            catname varchar(10),
            catdesc varchar(50));


        create table date(
            dateid smallint not null distkey sortkey,
            caldate date not null,
            day character(3) not null,
            week smallint not null,
            month character(5) not null,
            qtr character(5) not null,
            year smallint not null,
            holiday boolean default('N'));

        create table event(
            eventid integer not null distkey,
            venueid smallint not null,
            catid smallint not null,
            dateid smallint not null sortkey,
            eventname varchar(200),
            starttime timestamp);

        create table listing(

        listid integer not null distkey,
        sellerid integer not null,
        eventid integer not null,
        dateid smallint not null sortkey,
        numtickets smallint not null,
        priceperticket decimal(8,2),
        totalprice decimal(8,2),
        listtime timestamp);


""")

except psycopg2.Error as e:
        print('Error: issue creating table')
        print(e) 

In [56]:
try:
    cur.execute("""
    copy users from 's3://mabohassan-test/allusers_pipe.txt'
    credentials 'aws_iam_role=arn:aws:iam::777381563271:role/redshift-s3-acces'
    delimiter '|'
    region 'us-west-2'
    """)


except psycopg2.Error as e:
        print('Error: issue creating table')
        print(e) 

In [60]:
try:
    cur.execute("""
        select * from users;
    """)


except psycopg2.Error as e:
        print('Error: issue creating table')
        print(e) 

In [61]:
row=cur.fetchone()
print(row)


(2, 'PGL08LJI', 'Vladimir', 'Humphrey', 'Murfreesboro', 'SK', 'Suspendisse.tristique@nonnisiAenean.edu', '(783) 492-1886', None, None, None, True, True, None, None, True, False, True)


In [59]:
try:
    conn.close()
except psycopg2.Error as e:
    print(e)

In [ ]:
#### Delete Resources !!

redshift.delete_cluster( ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,  SkipFinalClusterSnapshot=True)